In [39]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# Define the path to the input image which will be the sample image of yourself on which it will train
IMAGE_FILE = 'image.jpg'
OUTPUT_LANDMARKS_FILE = 'face_landmarks.txt'

# For static images:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:

    # Read the image
    image = cv2.imread(IMAGE_FILE)
    
    # Convert the BGR image to RGB before processing
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Check if face landmarks are detected
    if results.multi_face_landmarks:
        annotated_image = image.copy()
        for face_landmarks in results.multi_face_landmarks:
            # Draw face mesh landmarks on the image
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_IRISES,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_iris_connections_style())
            
            # Save landmarks to a text file
            with open(OUTPUT_LANDMARKS_FILE, 'w') as f:
                for landmark in face_landmarks.landmark:
                    f.write(f'{landmark.x}, {landmark.y}, {landmark.z}\n')

        # Save the annotated image
        cv2.imwrite('annotated_image.png', annotated_image)
    else:
        print("No face landmarks detected.")


In [40]:
import cv2
import mediapipe as mp
import numpy as np

# Load the landmarks from the text file
def load_landmarks(file_path):
    with open(file_path, 'r') as f:
        landmarks = []
        for line in f:
            x, y, z = map(float, line.strip().split(', '))
            landmarks.append([x, y, z])
    return np.array(landmarks)

# Normalize landmarks by scaling and translating to a canonical space
def normalize_landmarks(landmarks):
    center = np.mean(landmarks, axis=0)
    landmarks -= center
    max_dist = np.max(np.linalg.norm(landmarks, axis=1))
    landmarks /= max_dist
    return landmarks

# Compare two sets of landmarks
def compare_landmarks(landmarks1, landmarks2, threshold=0.1):
    if len(landmarks1) != len(landmarks2):
        return False
    distance = np.linalg.norm(landmarks1 - landmarks2, axis=1)
    return np.mean(distance) < threshold

# Load and normalize the saved landmarks
saved_landmarks = normalize_landmarks(load_landmarks('face_landmarks.txt'))

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    message = "not same"
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())

        # Convert landmarks to numpy array and normalize
        detected_landmarks = np.array([(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark])
        normalized_landmarks = normalize_landmarks(detected_landmarks)

        # Compare the detected landmarks with the saved landmarks
        if compare_landmarks(normalized_landmarks, saved_landmarks):
            message = "same person"

    # Flip the image horizontally for a selfie-view display.
    image = cv2.flip(image, 1)
    
    # Display the message on the image
    cv2.putText(image, message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if message == "same person" else (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('MediaPipe Face Mesh', image)
    if cv2.waitKey(5) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()
